# Notebook to analyse sale history from DVF

### Objectif 

- Pour chaque bien, avoir une visibilité des opérations réalisées autour. Récupérer le maxiumu d'informations sur cette vente.
- Identifier des opérations d'achat-revente.

In [19]:
import pandas as pd
import folium

### Chargement des données

In [20]:
df = pd.read_csv("full.csv")

/var/folders/pz/pwb4fxrn0z73q85rm8lhqkh00000gn/T/ipykernel_79276/2471715599.py:1: DtypeWarning: Columns (8,10,12,14,17,18,20,22,24,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("full.csv")


In [21]:
df.columns

Index(['id_mutation', 'date_mutation', 'numero_disposition', 'nature_mutation',
       'valeur_fonciere', 'adresse_numero', 'adresse_suffixe',
       'adresse_nom_voie', 'adresse_code_voie', 'code_postal', 'code_commune',
       'nom_commune', 'code_departement', 'ancien_code_commune',
       'ancien_nom_commune', 'id_parcelle', 'ancien_id_parcelle',
       'numero_volume', 'lot1_numero', 'lot1_surface_carrez', 'lot2_numero',
       'lot2_surface_carrez', 'lot3_numero', 'lot3_surface_carrez',
       'lot4_numero', 'lot4_surface_carrez', 'lot5_numero',
       'lot5_surface_carrez', 'nombre_lots', 'code_type_local', 'type_local',
       'surface_reelle_bati', 'nombre_pieces_principales',
       'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale',
       'nature_culture_speciale', 'surface_terrain', 'longitude', 'latitude'],
      dtype='object')

In [27]:
to_value_counts = ['nature_mutation', 'nombre_lots', 'code_type_local', 'type_local',  'nombre_pieces_principales', 'code_nature_culture', 'nature_culture', 'code_nature_culture_speciale', 'nature_culture_speciale', 'surface_terrain']
#'surface_reelle_bati',

for r in to_value_counts : 
    print("===")
    print(df[r].value_counts())
    print()

# value_counts

===
nature_mutation
Vente                                 1475841
Vente en l'état futur d'achèvement      61729
Echange                                 22527
Vente terrain à bâtir                    3681
Adjudication                             2109
Expropriation                             756
Name: count, dtype: int64

===
nombre_lots
0      1102418
1       316709
2       122449
3        17347
4         4703
5         1311
6          750
7          336
8          175
10          97
9           84
12          58
11          44
13          31
14          19
15          15
16          11
20          10
17          10
18           9
19           7
22           4
21           3
25           3
29           2
42           2
44           2
33           2
34           2
23           2
30           2
161          2
39           2
27           2
28           2
32           2
60           1
58           1
105          1
72           1
46           1
236          1
75           1
51           1
4

In [30]:
df['surface_reelle_bati'].isna().mean() #.value_counts()

np.float64(0.6918047059859841)

In [32]:
df['surface_terrain'].isna().mean()

np.float64(0.3007787989988785)

In [10]:
# Sélectionner uniquement les colonnes de type catégoriel
# categorical_columns = df.select_dtypes(include=['object', 'category']).columns

# Appliquer .unique() à chaque colonne catégorielle
# unique_values = {col: df[col].unique() for col in categorical_columns}

In [102]:
#df.surface_reelle_bati.isna().mean()

In [14]:
filtered_df = df[df['code_postal'] == 77000]

In [15]:
filtered_df["type_local"].value_counts()

type_local
Dépendance                                  373
Appartement                                 235
Maison                                       89
Local industriel. commercial ou assimilé     44
Name: count, dtype: int64

### Affichage des données de ventes

In [103]:
# ---------------------------
# ADAPTER LE CODE AUX DONNÉES EXISTANTES
# ---------------------------
import pandas as pd
import folium
from folium.plugins import HeatMap

# 1. Choisir un code postal présent dans les données (ex: 1170 = Gex)
code_postal_cible = 44550 #77000 

# 2. Filtrer les données et calculer le prix/m²
filtered_df = df[df['code_postal'] == code_postal_cible].copy()
filtered_df = filtered_df.dropna(subset=['valeur_fonciere', 'surface_terrain', 'longitude', 'latitude'])
filtered_df = filtered_df[filtered_df['surface_terrain'] > 0]
#filtered_df = filtered_df[filtered_df["type_local"]=="Appartements"]
filtered_df['prix_m2'] = filtered_df['valeur_fonciere'] / filtered_df['surface_terrain']

# 3. Coordonnées du centre de la carte (ex: Gex - 46.332212, 6.058695)
latitude_centre = filtered_df['latitude'].mean()
longitude_centre = filtered_df['longitude'].mean()

# ---------------------------
# CRÉATION DE LA CARTE
# ---------------------------

m = folium.Map(
    location=[latitude_centre, longitude_centre],
    zoom_start=14,
    tiles="cartodbpositron"
)

# Heatmap des prix/m²
heat_data = np.array([[row['latitude'], row['longitude'], row['prix_m2']] for _, row in filtered_df.iterrows()])
HeatMap(
    heat_data,
    name='Prix au m²',
    #gradient={0.4: 'blue', 0.6: 'lime', 1: 'red'},  # Personnaliser les couleurs
    min_opacity=0.1,
    radius=30
).add_to(m)

# Marqueurs interactifs
for _, row in filtered_df.iterrows():
    popup_html = f"""
    <b>Adresse:</b> {row['adresse_numero']} {row['adresse_nom_voie']}<br>
    <b>Prix:</b> {row['valeur_fonciere']:,.0f} €<br>
    <b>Surface terrain:</b> {row['surface_terrain']} m²<br>
    <b>Prix/m²:</b> {row['prix_m2']:,.1f} €
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=popup_html,
        icon=folium.Icon(color='green', icon='euro', prefix='fa' ),
        tooltip=f"{row['prix_m2']:,.0f} €/m²"
    ).add_to(m)

# Contrôle des calques + Affichage
#folium.LayerControl().add_to(m)
display(m)

ValueError: data may not contain NaNs.

In [63]:
#import numpy as np
#data = (
#    np.random.normal(size=(100, 3)) * np.array([[1, 1, 1]]) + np.array([[latitude_centre, longitude_centre, 1]])
#).tolist()

In [64]:
#HeatMap(np.array(heat_data)).add_to(m)

In [65]:
#display(m)

In [78]:
m.save("map.html")